# Packages 📦
Run the following cell to load all needed packages.

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Data 📊
Lets get familiar with the data we will be working with. Start by loading the training data into a pandas Dataframe and extracting high level information.

In [30]:
df = pd.read_csv("../data/train.csv")
df.describe()

Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   181.066207   456.098091  ...   
min       1.000000  1872.000000   1950.000000     0.000000     0.000000  ...   
25%       5.000000  1954.000000   1967.000000     0.000000     0.000000  ...   
50%       5.000000  1973.000000   1994.000000     0.000000   383.500000  ...   
75%       6.000000  2000.000000   2004.000000   166.000000   712.250000  ...   
max       9.000000  2010.000000   2010.000000  1600.000000  5644.000000  ...   

        WoodDeckSF  OpenPorchSF  EnclosedPorch    3SsnPorch  ScreenPorch  \
count  1460.000000  1460.000000    1460.000000  1460.000000  1460.000000   
mean     94.244521    46.660274      21.954110     3.409589    15.060959   
std     125.338794    66.256028      61.119149    29.317331    55.757415   
min       0.000000     0.000000       0.000000     0.000000     0.000000   
25%       0.000000     0.000000       0.000000     0.000000     0.000000   
50%       0.000000    25.000000       0.000000     0.000000     0.000000   
75%     168.000000    68.000000       0.000000     0.000000     0.000000   
max     857.000000   547.000000     552.000000   508.000000   480.000000   

          PoolArea       MiscVal       MoSold       YrSold      SalePrice  
count  1460.000000   1460.000000  1460.000000  1460.000000    1460.000000  
mean      2.758904     43.489041     6.321918  2007.815753  180921.195890  
std      40.177307    496.123024     2.703626     1.328095   79442.502883  
min       0.000000      0.000000     1.000000  2006.000000   34900.000000  
25%       0.000000      0.000000     5.000000  2007.000000  129975.000000  
50%       0.000000      0.000000     6.000000  2008.000000  163000.000000  
75%       0.000000      0.000000     8.000000  2009.000000  214000.000000  
max     738.000000  15500.000000    12.000000  2010.000000  755000.000000  

[8 rows x 38 columns]

# Data 📊
- As we can observe from the result of the previous cell, some features are on a different scale than other features. This difference in scale makes it a ideal for us to normalize the data to bring all features to a small and common scale.
- Also, we can observe that some of the features are in a significantly higher scale than others. We will need to scale this features down before normalizing them. We will use *log* to scale these features down

# Feature & Label 🏷️
- Before scaling and normalizing the data, lets go ahead and split our feature data from our label data

In [31]:
y = df[["Id", "SalePrice"]]
X = df.drop(columns=["Id", "SalePrice"])

print(f"Label Dataset shape: {y.shape}")
print(f"Feature Dataset shape: {X.shape}")
y.head()
X.head()

Label Dataset shape: (1460, 2)
Feature Dataset shape: (1460, 79)


MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0          60       RL         65.0     8450   Pave   NaN      Reg   
1          20       RL         80.0     9600   Pave   NaN      Reg   
2          60       RL         68.0    11250   Pave   NaN      IR1   
3          70       RL         60.0     9550   Pave   NaN      IR1   
4          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC Fence  \
0         Lvl    AllPub    Inside  ...           0        0    NaN   NaN   
1         Lvl    AllPub       FR2  ...           0        0    NaN   NaN   
2         Lvl    AllPub    Inside  ...           0        0    NaN   NaN   
3         Lvl    AllPub    Corner  ...           0        0    NaN   NaN   
4         Lvl    AllPub       FR2  ...           0        0    NaN   NaN   

  MiscFeature MiscVal  MoSold  YrSold  SaleType  SaleCondition  
0         NaN       0       2    2008        WD         Normal  
1         NaN       0       5    2007        WD         Normal  
2         NaN       0       9    2008        WD         Normal  
3         NaN       0       2    2006        WD        Abnorml  
4         NaN       0      12    2008        WD         Normal  

[5 rows x 79 columns]

# NaN Column Values 🚫
- Lets extract columns that have missing values. We will need to populate these columns with the appropriate data otherwise we will face issues when training the model.
- For this project, we will update **Null** values with the **median** ONLY for numerical columns.

In [32]:
na_columns = {col: i for i, col in enumerate(X.columns) if X[col].isna().any()}

print(f"\nAmount of columns with NaN values: {len(na_columns.keys())}")
print(f"\nColumns with NaN values: {na_columns.keys()}")

# Extracts columns from na_columns whose datatype is not numeric
nan_cols = {col: idx for col, idx in na_columns.items() if X[col].dtype == type(object)}
print(f"\nAmount of Non-numerical columns with Null values: {len(nan_cols.keys())}")
print(f"Columns of type object with null values: {nan_cols.keys()}")

# Extract numerical cols with Null values
num_cols_na = {col: idx for col, idx in na_columns.items() if nan_cols.get(col, False) is False}
print(f"\nAmount of numerical columns with Null values: {len(num_cols_na.keys())}")
print(f"Columns of type object with null values: {num_cols_na.keys()}")

# Extract the median for each column
na_cols_median_vals = {col: X[col].median() for col in num_cols_na.keys()}
print(na_cols_median_vals)

#Fill NaNs
for col in num_cols_na:
    X[col] = X[col].fillna(na_cols_median_vals[col])


# Fill cols with 'NA'
# > if previous logic does not work, try this: For non-numeric columns, populate NULL values with the most frequent used value
for col in nan_cols:
    X[col] = X[col].fillna("Missing")
    #col_counts = col_s.value_counts()
    #print(col_counts)

# Check for NaNs to ensure all columns are populated
na_columns = {col: i for i, col in enumerate(X.columns) if X[col].isna().any()}
print(f"\nColumns with Nulls: {len(na_columns.keys())}")
X.head()


Amount of columns with NaN values: 19

Columns with NaN values: dict_keys(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'])

Amount of Non-numerical columns with Null values: 16
Columns of type object with null values: dict_keys(['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'])

Amount of numerical columns with Null values: 3
Columns of type object with null values: dict_keys(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'])
{'LotFrontage': np.float64(69.0), 'MasVnrArea': np.float64(0.0), 'GarageYrBlt': np.float64(1980.0)}

Columns with Nulls: 0


MSSubClass MSZoning  LotFrontage  LotArea Street    Alley LotShape  \
0          60       RL         65.0     8450   Pave  Missing      Reg   
1          20       RL         80.0     9600   Pave  Missing      Reg   
2          60       RL         68.0    11250   Pave  Missing      IR1   
3          70       RL         60.0     9550   Pave  Missing      IR1   
4          60       RL         84.0    14260   Pave  Missing      IR1   

  LandContour Utilities LotConfig  ... ScreenPorch PoolArea   PoolQC    Fence  \
0         Lvl    AllPub    Inside  ...           0        0  Missing  Missing   
1         Lvl    AllPub       FR2  ...           0        0  Missing  Missing   
2         Lvl    AllPub    Inside  ...           0        0  Missing  Missing   
3         Lvl    AllPub    Corner  ...           0        0  Missing  Missing   
4         Lvl    AllPub       FR2  ...           0        0  Missing  Missing   

  MiscFeature MiscVal  MoSold  YrSold  SaleType  SaleCondition  
0     Missing       0       2    2008        WD         Normal  
1     Missing       0       5    2007        WD         Normal  
2     Missing       0       9    2008        WD         Normal  
3     Missing       0       2    2006        WD        Abnorml  
4     Missing       0      12    2008        WD         Normal  

[5 rows x 79 columns]

# Train & Validation Datasets 📈
- In this part of the notebook, we will normalize our dataset as well as splittling between between **train** and **validation** datasets.
- For features that are on a larger scale than others, we will scale them down using **log** before normalizing them.
- For normalization, we are going to utilize scikit-learn **StandardScaler** which allows us to normalize our features with **z-score** normalization.

In [36]:
# Lets start by splittling our dataset into train and validation
print(f"\nFeatures shape before splitting: {X.shape}")
print(f"\nLabel shape before splitting: {y.shape}")
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.3, shuffle=True)

print(f"\nTrain Features shape: {X_train.shape}")
print(f"Train Labels shape: {y_train.shape}")
print(f"\nTrain Features shape: {X_cv.shape}")
print(f"Train Labels shape: {y_cv.shape}")

# Now that we splitted our Datasets into train and cv, lets go ahead and scale and normalize our features



Features shape before splitting: (1460, 79)

Label shape before splitting: (1460, 2)

Train Features shape: (1022, 79)
Train Labels shape: (1022, 2)

Train Features shape: (438, 79)
Train Labels shape: (438, 2)
